In [50]:
import pandas as pd
import numpy as np

# Function to calculate RSI
def calculate_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# Function to calculate MACD
def calculate_macd(series, short_period=12, long_period=26, signal_period=9):
    short_ema = series.ewm(span=short_period, adjust=False).mean()
    long_ema = series.ewm(span=long_period, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_period, adjust=False).mean()
    return macd, signal

# Function to calculate OBV
def calculate_obv(data):
    obv = [0]  # OBV starts at 0
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['Close'].iloc[i - 1]:
            obv.append(obv[-1] + data['Volume'].iloc[i])
        elif data['Close'].iloc[i] < data['Close'].iloc[i - 1]:
            obv.append(obv[-1] - data['Volume'].iloc[i])
        else:
            obv.append(obv[-1])
    data['OBV'] = obv
    return data

# Function to calculate Supertrend
def calculate_supertrend(data, atr_period=10, multiplier=3):
    high_low = data['High'] - data['Low']
    high_close = abs(data['High'] - data['Close'].shift())
    low_close = abs(data['Low'] - data['Close'].shift())
    tr = high_low.combine(high_close, max).combine(low_close, max)
    atr = tr.rolling(window=atr_period).mean()

    data['Upper_Band'] = ((data['High'] + data['Low']) / 2) + (multiplier * atr)
    data['Lower_Band'] = ((data['High'] + data['Low']) / 2) - (multiplier * atr)

    data['Supertrend'] = np.nan
    in_uptrend = True

    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['Upper_Band'].iloc[i-1]:
            in_uptrend = True
        elif data['Close'].iloc[i] < data['Lower_Band'].iloc[i-1]:
            in_uptrend = False

        if in_uptrend:
            data.loc[i, 'Supertrend'] = data['Lower_Band'].iloc[i]
        else:
            data.loc[i, 'Supertrend'] = data['Upper_Band'].iloc[i]
    
    return data


# Function to apply the strategy and backtest
import pandas as pd
import pandas_ta as ta
import numpy as np

# Function to calculate Pivot Points and levels
def calculate_pivot_points(data):
    data['Pivot'] = (data['High'].shift(1) + data['Low'].shift(1) + data['Close'].shift(1)) / 3
    data['R1'] = (data['Pivot'] * 2) - data['Low'].shift(1)
    data['S1'] = (data['Pivot'] * 2) - data['High'].shift(1)
    data['R2'] = data['Pivot'] + (data['R1'] - data['S1'])
    data['S2'] = data['Pivot'] - (data['R1'] - data['S1'])
    data['R3'] = data['Pivot'] + (data['R2'] - data['S2'])
    data['S3'] = data['Pivot'] - (data['R2'] - data['S2'])
    return data

# Function to generate buy/sell signals
def generate_signals(data, start_time='09:20:00', end_time='15:00:00'):
    data['VMA'] = data['Volume'].rolling(window=10).mean()
    
    # Time filtering
    data['Time'] = data['Datetime'].dt.time
    start_time = pd.to_datetime(start_time).time()
    end_time = pd.to_datetime(end_time).time()

    # Generate buy signals
    data['Buy_Signal'] = np.where(
        # (data['RSI'] < 30) & 
        # (data['OBV'].diff() > 0) &
        # (data['Close'] > data['Pivot']) &
        # (data['Volume'] > data['VMA']) &
        (data['Close'] > data['Supertrend']) &
        (data['Time'] >= start_time) & (data['Time'] <= end_time ),
        1, 0)

    # Generate sell signals
    data['Sell_Signal'] = np.where(
        # (data['RSI'] > 70) & 
        # (data['OBV'].diff() < 0) &
        # (data['Close'] < data['Pivot']) &
        # (data['Volume'] > data['VMA']) &
        (data['Close'] < data['Supertrend']) &
        (data['Time'] >= start_time) & (data['Time'] <= end_time),
        1, 0)

    return data

# Function to apply the strategy and backtest
def apply_strategy(file_path, initial_capital=100000):
    stock_data = pd.read_csv(file_path)
    stock_data['Datetime'] = pd.to_datetime(stock_data['Datetime'])
    

    # Calculate Pivot Points
    stock_data = calculate_pivot_points(stock_data)

    # Calculate indicators using pandas_ta
    stock_data['RSI'] = ta.rsi(stock_data['Close'], length=14)
    stock_data['OBV'] = ta.obv(stock_data['Close'], stock_data['Volume'])
    
    supertrend = ta.supertrend(stock_data['High'], stock_data['Low'], stock_data['Close'], length=10, multiplier=3)
    stock_data['Supertrend'] = supertrend['SUPERT_10_3.0']

    stock_data.dropna(inplace=True)
    print(stock_data.head())
    # Generate signals
    stock_data = generate_signals(stock_data)

    # Backtest the strategy
    capital = initial_capital
    position = 0
    stock_data['Strategy_Returns'] = 0

    for i in range(1, len(stock_data)):
        if stock_data['Buy_Signal'].iloc[i] == 1 and position == 0:
            position = capital / stock_data['Close'].iloc[i]
            capital = 0
        elif stock_data['Sell_Signal'].iloc[i] == 1 and position > 0:
            capital = position * stock_data['Close'].iloc[i]
            position = 0
        # Force exit at 15:15
        elif stock_data['Time'].iloc[i] == pd.to_datetime('15:15:00').time() and position > 0:
            capital = position * stock_data['Close'].iloc[i]
            position = 0
        stock_data.loc[i, 'Strategy_Returns'] = capital + position * stock_data['Close'].iloc[i]

    total_returns = (capital + position * stock_data['Close'].iloc[-1]) - initial_capital
    percentage_return = (total_returns / initial_capital) * 100

    return total_returns, percentage_return




In [51]:
nifty_200 = pd.read_csv("ind_nifty200list.csv")["Symbol"].to_list()

In [52]:
for ticker in nifty_200:
    # try:
        result = apply_strategy(f"nifty_200/{ticker}.csv")
        print(f"{ticker}: {result}")
    # except:
    #     print(f"Error: {ticker}")

              Datetime     Open     High     Low    Close  Volume  \
14 2020-07-06 09:15:00   982.00  1017.00  982.00  1010.70   85601   
15 2020-07-06 10:15:00  1010.55  1012.15  992.45   998.55  103625   
16 2020-07-06 11:15:00   998.75   999.00  993.05   993.30   43105   
17 2020-07-06 12:15:00   993.30   999.50  990.00   991.00   33653   
18 2020-07-06 13:15:00   991.05   991.85  976.20   982.50   62657   

          Pivot           R1          S1           R2          S2  \
14   980.833333   986.666667  973.666667   993.833333  967.833333   
15  1003.233333  1024.466667  989.466667  1038.233333  968.233333   
16  1001.050000  1009.650000  989.950000  1020.750000  981.350000   
17   995.116667   997.183333  991.233333  1001.066667  989.166667   
18   993.500000   997.000000  987.500000  1003.000000  984.000000   

             R3          S3        RSI       OBV  Supertrend  
14  1006.833333  954.833333  81.696857   61813.0   951.65244  
15  1073.233333  933.233333  68.517861  -418

KeyboardInterrupt: 